In [1]:
console.log(require("child_process").execSync(
    "cd ../../; pwd"
).toString())

/Users/waltmayf/Documents/repos/amplify-langgraph-template



In [2]:
const { z } = require('zod');
const { tool } = require('@langchain/core/tools');

const { ChatBedrockConverse } = require("@langchain/aws");
const { HumanMessage, AIMessage, SystemMessage, BaseMessage } = require("@langchain/core/messages");
const { createReactAgent } = require("@langchain/langgraph/prebuilt");
const { Calculator } = require("@langchain/community/tools/calculator");
const { EventEmitter } = require("events");

const { renderHumanMessage, renderAIMessage, renderToolMessage } = require('./renderMessages.cjs');

process.env.AWS_DEFAULT_REGION='us-east-1'

let llm
let tools
let agent
let main
let myNewToolSchema
let myNewToolDefinitoin
let agentFinalState
let toolMessageResponse

'us-east-1'

In [3]:
//Define a ZOD object with the tool argument schema
myNewToolSchema = z.object({
    title: z.string(),
    description: z.string(),
    buttonTextBeforeClick: z.string(),
    buttonTextAfterClick: z.string(),
})

myNewToolDefinition = tool(
    async (userInputToolArgs) => {
        return {
            success: true,
            ...userInputToolArgs
        }
    },
    {
        name: "userInputTool",
        description: `
Use this tool to send emails or add items to a work management system.
The messages should never request information.
They should only inform someone besides the user about an action they should take (including to review an item from the chat).
`,
        schema: myNewToolSchema,
    }
);

DynamicStructuredTool {
  lc_serializable: false,
  lc_kwargs: {
    name: 'userInputTool',
    description: '\n' +
      'Use this tool to send emails or add items to a work management system.\n' +
      'The messages should never request information.\n' +
      'They should only inform someone besides the user about an action they should take (including to review an item from the chat).\n',
    schema: ZodObject {
      spa: [Function: bound safeParseAsync] AsyncFunction,
      _def: [Object],
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [F

In [4]:
// Initialize Bedrock LLM
llm = new ChatBedrockConverse({
    model: "us.anthropic.claude-3-5-haiku-20241022-v1:0"
});

// Define available tools
tools = [
    myNewToolDefinition,
    
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

CompiledStateGraph {
  lc_serializable: false,
  lc_kwargs: {
    checkpointer: undefined,
    interruptAfter: undefined,
    interruptBefore: undefined,
    autoValidate: false,
    nodes: {
      __start__: [PregelNode],
      agent: [PregelNode],
      tools: [PregelNode]
    },
    channels: {
      messages: [BinaryOperatorAggregate],
      structuredResponse: [LastValue],
      __start__: [EphemeralValue],
      agent: [EphemeralValue],
      tools: [EphemeralValue],
      'branch:__start__:__self__:agent': [EphemeralValue],
      'branch:__start__:__self__:tools': [EphemeralValue],
      'branch:agent:__self__:agent': [EphemeralValue],
      'branch:agent:__self__:tools': [EphemeralValue],
      'branch:tools:__self__:agent': [EphemeralValue],
      'branch:tools:__self__:tools': [EphemeralValue],
      '__start__:agent': [EphemeralValue],
      'branch:agent:condition:tools': [EphemeralValue]
    },
    inputChannels: '__start__',
    outputChannels: [ 'messages', 'structuredRe

In [5]:
main = async () => {
    const agentFinalState = await agent.invoke(
      { messages: [new HumanMessage("I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.")] }
    );
    console.log(agentFinalState)

    toolMessageResponse = agentFinalState.messages.find(msg => msg.constructor.name === 'ToolMessage');
    
    return agentFinalState
}

agentFinalState = main()

Promise { <pending> }

{
  messages: [
    HumanMessage {
      "id": "1ec9a1f7-1afc-415f-a3d9-d5d6f4ab9904",
      "content": "I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "17fcb715-7465-4942-9116-f47a135e8395",
      "content": [
        {
          "type": "text",
          "text": "I'll help you send a notification to Jeff about purchasing a new downhole pump for the Fruitland Coal well in New Mexico."
        }
      ],
      "additional_kwargs": {},
      "response_metadata": {
        "$metadata": {
          "httpStatusCode": 200,
          "requestId": "17fcb715-7465-4942-9116-f47a135e8395",
          "attempts": 1,
          "totalRetryDelay": 0
        },
        "metrics": {
          "latencyMs": 3669
        },
        "stopReason": "tool_use",
        "usage": {
          "cacheReadInputTokenCount": 0,
          "cacheReadInputTokens": 0,
          "cacheWri

In [7]:
$$.html(
    (async () => {
        const result = await agent.invoke(
            { messages: [new HumanMessage("I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.")] }
        );
        
        const toolMessage = result.messages.find(msg => msg.constructor.name === 'ToolMessage');
        const toolContent = JSON.parse(toolMessage.content);
        
        const html = `
            <div style="border: 1px solid #ccc; padding: 15px; border-radius: 5px;">
                <h3 style="margin-top: 0;">${toolContent.title}</h3>
                <p style="white-space: pre-wrap;">${toolContent.description}</p>
                <button 
                    onclick="this.textContent='${toolContent.buttonTextAfterClick}'; this.disabled=true; this.style.backgroundColor='#28a745';"
                    style="padding: 8px 16px; background-color: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer;">
                    ${toolContent.buttonTextBeforeClick}
                </button>
            </div>
        `;
        
        return html;  // You can still return the raw data if needed
    })()
)

Downhole Pump Purchase Request - Fruitland Coal Well, NM 
 I need to purchase a new downhole pump for our Fruitland Coal well located in New Mexico. Please provide a quote and availability for the required pump.

Key Details:
- Location: Fruitland Coal well, New Mexico
- Equipment Needed: Downhole Pump
- Action Required: Provide pricing and availability information 
 
 Send Request to Jeff

In [11]:
$$.html(
    (async () => {
        
        const result = await agent.invoke(
            { messages: [new HumanMessage("I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.")] }
        );
    
        // Render all messages in the conversation
        const conversationHtml = `
            <div style="font-family: system-ui, -apple-system, sans-serif; max-width: 800px; margin: 0 auto;">
                ${result.messages.map(message => {
                    switch(message.constructor.name) {
                        case 'HumanMessage':
                            return renderHumanMessage(message);
                        case 'AIMessage':
                            return renderAIMessage(message);
                        case 'ToolMessage':
                            return renderToolMessage(message);
                        default:
                            return '';
                    }
                }).join('\n')}
            </div>
        `;
        
        return conversationHtml
    })()
)

User 
 I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico. 
 


 
 Assistant 
 I'll help you create a notification for Jeff about the downhole pump purchase for the Fruitland Coal well in New Mexico. 
 


 
 Downhole Pump Purchase Request - Fruitland Coal Well 
 We need to purchase a new downhole pump for our Fruitland Coal well located in New Mexico. Please provide a quote and availability for the required pump.

Key Details:
- Location: Fruitland Coal well, New Mexico
- Item Needed: Downhole Pump
- Action Required: Provide pricing and availability information 
 
 Send Request to Jeff
 
 
 

 
 Assistant 
 I've sent a notification to Jeff requesting a quote and availability for the downhole pump for your Fruitland Coal well in New Mexico. The message includes the key details about the location and the specific item needed. Jeff should respond with pricing and availability information soon.

Is there anything else you would like me to help you with?